# COMP90042 Project 2018: Question Answering

Group Name: Strong

Team member:

Yunjie Jia 864538

qiangtou Guang 123456

Huorui QIang 234567

## General info

<b>Due date</b>: Due date: 11pm, Sunday 27th May, 2018



1. extract the document details---the document ids and text content---from the documents.json

In [32]:
import nltk
import json
from collections import defaultdict, Counter
from math import log
# nltk.download()

stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

# tokenise the documents, remove stop-words and stem the words to form our bag-of-words representation
def extract_terms(sentence):
    terms = set()
    for token in nltk.word_tokenize(sentence):
        if token not in stopwords:
            terms.add(stemmer.stem(token.lower()))
    return terms

# docid and text

def tokenize_sentence(docid):
    paras = {}
    with open('documents.json') as json_data:
        document_set = json.load(json_data) 
        para_id = 0
        for each_sentence in document_set[docid]['text']: 
            terms = extract_terms(each_sentence)
            paras[para_id] = terms
            para_id += 1
            
        docs[docid] = new_tokend_sentence
    return paras

def extract_term_freqs(sentence):
    tfs = Counter()
    for token in sentence:
        if token not in stopwords: 
            tfs[stemmer.stem(token.lower())] += 1
    return tfs


def compute_para_freqs(para_term_freqs):
    dfs = Counter()
    for tfs in para_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += 1
    return dfs  


def get_para_term_freqs(docid):
    para_term_freqs = {}
    current_doc = tokenize_sentence(docid)
    for para_id, each_para in current_doc.items():
        term_freqs = extract_term_freqs(each_para)
        para_term_freqs[para_id] = term_freqs
    M = len(para_term_freqs) 
    return para_term_freqs, M

def get_para_freqs(docid):
    para_term_freqs, M = get_para_term_freqs(docid)
    doc_freqs = compute_para_freqs(para_term_freqs)
    return doc_freqs

def get_inverted_index(docid):
    vsm_inverted_index = defaultdict(list)
    doc_term_freqs, M = get_para_term_freqs(docid)
    doc_freqs = get_para_freqs(docid)
    
    for para_id, term_freqs in doc_term_freqs.items():
        N = sum(term_freqs.values())
        length = 0

        # find tf*idf values and accumulate sum of squares 
        tfidf_values = []
        for term, count in term_freqs.items():
            tfidf = float(count) / N * log(M / float(doc_freqs[term]))
            tfidf_values.append((term, tfidf))
            length += tfidf ** 2

        # normalise documents by length and insert into index
        length = length ** 0.5
        for term, tfidf in tfidf_values:
            # note the inversion of the indexing, to be term -> (doc_id, score)
            vsm_inverted_index[term].append([para_id, tfidf / length])

    # ensure posting lists are in sorted order (less important here cf above)
    for term, para_ids in vsm_inverted_index.items():
        para_ids.sort()
        
    return vsm_inverted_index
        
        
def query_vsm(query, index, k=1):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)

results = query_vsm([stemmer.stem(term.lower()) for term in "In what year was the musical Broadway produced?".split()], get_inverted_index(409))
results

[(13, 0.2971890218996128)]

In [ ]:
# question,text, answer_paragraph, docid    
with open('devel.json') as json_data:
    develop_set = json.load(json_data)
    for i in range(len(develop_set)):
        question = develop_set[i]['question']
        doc_id = develop_set[i]['docid']
        para_in_json = develop_set[i]['answer_paragraph']
        answer_in_json = develop_set[i]['text']
        para_in_information_retrival = query_vsm([stemmer.stem(term.lower()) for term in question.split()], get_inverted_index(doc_id))
        print('answer in json:',para_in_json)
        print ('my model:',para_in_information_retrival)
    



('answer in json:', 5)
('my model:', [(0, 0.21205106088529624)])
('answer in json:', 22)
('my model:', [(22, 0.5898288907175065)])
('answer in json:', 16)
('my model:', [(16, 0.3035955689787876)])
('answer in json:', 4)
('my model:', [(11, 0.18904260619481944)])
('answer in json:', 2)
('my model:', [(4, 0.13816767984914657)])
('answer in json:', 1)
('my model:', [(1, 0.9408575391076255)])
('answer in json:', 25)
('my model:', [(25, 0.618668165290386)])
('answer in json:', 0)
('my model:', [(17, 0.05250482152558906)])
('answer in json:', 20)
('my model:', [(20, 0.8250910914934861)])
('answer in json:', 22)
('my model:', [(22, 0.5564212904502104)])
('answer in json:', 23)
('my model:', [(23, 0.43775736715608016)])
('answer in json:', 20)
('my model:', [(20, 0.33804308749657813)])
('answer in json:', 18)
('my model:', [(18, 0.4753295307717425)])
('answer in json:', 0)
('my model:', [(8, 0.21135705826650403)])
('answer in json:', 25)
('my model:', [(25, 0.12312864483811484)])
('answer in j

('answer in json:', 38)
('my model:', [(38, 0.7650878558103866)])
('answer in json:', 14)
('my model:', [(14, 0.3002324769077234)])
('answer in json:', 10)
('my model:', [(10, 0.2462792930227979)])
('answer in json:', 39)
('my model:', [(18, 0.003374496047918918)])
('answer in json:', 6)
('my model:', [(29, 0.3001153700593256)])
('answer in json:', 10)
('my model:', [(10, 0.20418336908008017)])
('answer in json:', 4)
('my model:', [(4, 0.29220670032916335)])
('answer in json:', 10)
('my model:', [(10, 0.4356824245391927)])
('answer in json:', 21)
('my model:', [(21, 0.622781485721836)])
('answer in json:', 2)
('my model:', [(2, 0.4521306016530998)])
('answer in json:', 30)
('my model:', [(30, 0.2408907133224695)])
('answer in json:', 4)
('my model:', [(4, 0.9644869387604069)])
('answer in json:', 23)
('my model:', [(23, 0.2781510971243074)])
('answer in json:', 38)
('my model:', [(38, 0.2181921084325817)])
('answer in json:', 32)
('my model:', [(1, 0.12305339420162527)])
('answer in js

('answer in json:', 38)
('my model:', [(38, 0.34021560996643463)])
('answer in json:', 45)
('my model:', [(45, 0.5513896238074809)])
('answer in json:', 27)
('my model:', [(31, 0.36894470573384214)])
('answer in json:', 40)
('my model:', [(40, 0.45453435302434114)])
('answer in json:', 12)
('my model:', [(12, 0.3431447157674309)])
('answer in json:', 22)
('my model:', [(22, 0.09196665668552081)])
('answer in json:', 7)
('my model:', [(1, 0.095529667558261)])
('answer in json:', 29)
('my model:', [(29, 0.6585044462735344)])
('answer in json:', 9)
('my model:', [(9, 0.5164773922397339)])
('answer in json:', 3)
('my model:', [(3, 0.2004376392065651)])
('answer in json:', 1)
('my model:', [(1, 0.5875909560296427)])
('answer in json:', 15)
('my model:', [(15, 0.3720374145171155)])
('answer in json:', 42)
('my model:', [(42, 0.479594724207865)])
('answer in json:', 0)
('my model:', [(38, 0.2585867741587115)])
('answer in json:', 38)
('my model:', [(38, 0.5240068511786763)])
('answer in json:

('answer in json:', 24)
('my model:', [(23, 0.2702060157624433)])
('answer in json:', 1)
('my model:', [(1, 0.06471325872940913)])
('answer in json:', 18)
('my model:', [(18, 0.22685240093225614)])
('answer in json:', 6)
('my model:', [(5, 0.20500242545347885)])
('answer in json:', 6)
('my model:', [(4, 0.26009716826545193)])
('answer in json:', 15)
('my model:', [(13, 0.27161587444465185)])
('answer in json:', 12)
('my model:', [(17, 0.32838161145170264)])
('answer in json:', 23)
('my model:', [(23, 0.3591914837952641)])
('answer in json:', 29)
('my model:', [(24, 0.14057192466957527)])
('answer in json:', 19)
('my model:', [(22, 0.19932949944233785)])
('answer in json:', 6)
('my model:', [(4, 0.26009716826545193)])
('answer in json:', 31)
('my model:', [(31, 0.5812935155673278)])
('answer in json:', 21)
('my model:', [(21, 0.5537977467518473)])
('answer in json:', 24)
('my model:', [(24, 0.510108891748161)])
('answer in json:', 16)
('my model:', [(10, 0.27986001296535434)])
('answer 

('answer in json:', 0)
('my model:', [(0, 0.674065750281434)])
('answer in json:', 23)
('my model:', [(23, 0.6965453522207647)])
('answer in json:', 14)
('my model:', [(13, 0.25354434970584316)])
('answer in json:', 8)
('my model:', [(8, 0.40669252255698773)])
('answer in json:', 0)
('my model:', [(0, 0.2941518117543859)])
('answer in json:', 22)
('my model:', [(22, 0.4547023420297379)])
('answer in json:', 31)
('my model:', [(30, 0.4715277784646439)])
('answer in json:', 28)
('my model:', [(28, 0.40452697027975015)])
('answer in json:', 2)
('my model:', [(2, 0.5789314954855069)])
('answer in json:', 27)
('my model:', [(7, 0.23632706306603973)])
('answer in json:', 19)
('my model:', [(19, 1.3163207431208603)])
('answer in json:', 22)
('my model:', [(22, 0.2941028910407564)])
('answer in json:', 31)
('my model:', [(9, 0.18860369673773955)])
('answer in json:', 0)
('my model:', [(28, 0.3252167809229425)])
('answer in json:', 19)
('my model:', [(19, 0.6475740541364969)])
('answer in json:

...

('answer in json:', 9)
('my model:', [(9, 0.45183136636751864)])
('answer in json:', 25)
('my model:', [(25, 0.594183076606616)])
('answer in json:', 26)
('my model:', [(26, 0.3718419748622341)])
('answer in json:', 33)
('my model:', [(30, 0.1573847413356346)])
('answer in json:', 34)
('my model:', [(34, 0.23989989257866526)])
('answer in json:', 17)
('my model:', [(17, 0.8217611482086696)])
('answer in json:', 25)
('my model:', [(25, 0.5360166826525592)])
('answer in json:', 37)
('my model:', [(22, 0.4071638810999386)])
('answer in json:', 33)
('my model:', [(33, 0.4697621093141998)])
('answer in json:', 16)
('my model:', [(23, 0.23958099221187856)])
('answer in json:', 5)
('my model:', [])
('answer in json:', 14)
('my model:', [(14, 0.7522824208308396)])
('answer in json:', 2)
('my model:', [(2, 0.423918142816114)])
('answer in json:', 5)
('my model:', [(5, 0.5221369288783102)])
('answer in json:', 4)
('my model:', [])
('answer in json:', 29)
('my model:', [(29, 0.6393579162373223)])

('answer in json:', 9)
('my model:', [(9, 0.3338023680180432)])
('answer in json:', 25)
('my model:', [(25, 0.16932093228673015)])
('answer in json:', 21)
('my model:', [(21, 0.3209831254684342)])
('answer in json:', 24)
('my model:', [(24, 0.40131481839154043)])
('answer in json:', 23)
('my model:', [(20, 0.16377411085382498)])
('answer in json:', 30)
('my model:', [(7, 0.17411906113527909)])
('answer in json:', 0)
('my model:', [(28, 0.1593799274443131)])
('answer in json:', 9)
('my model:', [(9, 0.31137696881305454)])
('answer in json:', 0)
('my model:', [(28, 0.1593799274443131)])
('answer in json:', 9)
('my model:', [(9, 0.3338023680180432)])
('answer in json:', 7)
('my model:', [(7, 0.17411906113527909)])
('answer in json:', 0)
('my model:', [(0, 0.4379651008605808)])
('answer in json:', 4)
('my model:', [(4, 0.47045745110252496)])
('answer in json:', 12)
('my model:', [])
('answer in json:', 25)
('my model:', [(25, 0.22646754597437435)])
('answer in json:', 25)
('my model:', [(2

('answer in json:', 28)
('my model:', [(28, 0.3526096724896137)])
('answer in json:', 6)
('my model:', [(23, 0.16618864565821792)])
('answer in json:', 35)
('my model:', [(35, 0.34865965068815463)])
('answer in json:', 9)
('my model:', [(26, 0.28195952901066557)])
('answer in json:', 11)
('my model:', [(24, 0.1533824568214754)])
('answer in json:', 11)
('my model:', [(12, 0.3932616277568661)])
('answer in json:', 42)
('my model:', [(42, 0.3081595544414646)])
('answer in json:', 34)
('my model:', [(34, 0.3524561307731652)])
('answer in json:', 5)
('my model:', [(17, 0.14356387298813253)])
('answer in json:', 10)
('my model:', [(10, 0.28182772634807873)])
('answer in json:', 8)
('my model:', [(8, 0.48217922942441593)])
('answer in json:', 11)
('my model:', [(11, 0.4011387943489497)])
('answer in json:', 39)
('my model:', [(39, 0.5363826239567695)])
('answer in json:', 11)
('my model:', [(15, 0.2175436032056237)])
('answer in json:', 1)
('my model:', [(1, 0.34320683391391893)])
('answer i

('answer in json:', 57)
('my model:', [])
('answer in json:', 64)
('my model:', [(28, 0.12104715853510492)])
('answer in json:', 6)
('my model:', [(6, 0.30634062082567093)])
('answer in json:', 42)
('my model:', [(30, 0.1287475819898961)])
('answer in json:', 5)
('my model:', [(5, 0.24325257784455845)])
('answer in json:', 3)
('my model:', [(3, 0.1352783331642793)])
('answer in json:', 37)
('my model:', [(37, 0.3959505127566003)])
('answer in json:', 8)
('my model:', [(72, 0.2476373226115282)])
('answer in json:', 66)
('my model:', [(66, 0.324727954474164)])
('answer in json:', 13)
('my model:', [(13, 0.34850087914742756)])
('answer in json:', 54)
('my model:', [(54, 0.26815809428607645)])
('answer in json:', 67)
('my model:', [(67, 0.32156560261697886)])
('answer in json:', 36)
('my model:', [(38, 0.12472588816567588)])
('answer in json:', 12)
('my model:', [(22, 0.1971688333851088)])
('answer in json:', 6)
('my model:', [(71, 0.13809053871063542)])
('answer in json:', 18)
('my model:

('answer in json:', 46)
('my model:', [(46, 0.4362298118834491)])
('answer in json:', 6)
('my model:', [(6, 0.20864038249145092)])
('answer in json:', 33)
('my model:', [(33, 0.27745093263564014)])
('answer in json:', 10)
('my model:', [(10, 0.22369265511012643)])
('answer in json:', 45)
('my model:', [(55, 0.16260954330137445)])
('answer in json:', 74)
('my model:', [(69, 0.13076393425981345)])
('answer in json:', 37)
('my model:', [(37, 0.2943717865207496)])
('answer in json:', 45)
('my model:', [(43, 0.20215563723533156)])
('answer in json:', 4)
('my model:', [(33, 0.17311488168502348)])
('answer in json:', 23)
('my model:', [(23, 0.21457561297270106)])
('answer in json:', 65)
('my model:', [(74, 0.10646820785382927)])
('answer in json:', 72)
('my model:', [(71, 0.36866176990542804)])
('answer in json:', 54)
('my model:', [(54, 0.23377424025165694)])
('answer in json:', 58)
('my model:', [(58, 0.25694555094750027)])
('answer in json:', 1)
('my model:', [(7, 0.34934772596772984)])
('

In [ ]:
with open('training.json') as json_data:
    train_set = json.load(json_data)
    

    
# question,text, answer_paragraph, docid    
with open('devel.json') as json_data:
    develop_set = json.load(json_data)
    
# question, docid, id: used when creating your Kaggle submissions
with open('testing.json') as json_data:
    test_set = json.load(json_data)    